# BRIEF : DATABASE CRUD

## Importation des librairies nécessaires et connexions à la base de données

In [32]:
import mysql.connector as mysql
import pandas as pd
import datetime as datetime

user = 'antan'
password = 'example'
host = 'localhost'
port = '3307'
database = 'CountriesDatas'

bdd = mysql.connect(user=user,password=password,host=host,port=port,database=database)
cursor = bdd.cursor()

### Ouverture du fichier texte

In [33]:
try:
    data_country = open("country.txt", "r") #ex : /dossier1/dossier2/mon_beau_fichier.txt
except Exception as e:
    print(e)

### Suppression de '__****__' dans le fichier texte pour faciliter sa lecture

In [34]:
countries = []
for i in data_country:
    i = i.replace('__****__ ', '')
    countries.append(i.strip('\n').split(" ",1))

### Insertion du fichier texte dans la table 'countries' de la BDD

In [35]:
for country in countries:
    code_country = country[0]
    name_country = country[1]
    #print(code_country)
    #print(name_country)
    sql = f"""INSERT INTO countries (code, name) VALUES ("{code_country}", "{name_country}") ON DUPLICATE KEY UPDATE code = '{code_country}' """
    
    cursor.execute(sql)

bdd.commit()


<center><img src="capture_countries.png" width="300" height="500"></center>

### Affichage du fichier csv 'population' sous forme de dataframe

In [36]:
try:
    # la ligne ci-dessous renvoit un dataframe
    data_population = pd.read_csv('population.csv')  #ex : /dossier1/dossier2/mon_beau_fichier.csv
except Exception as e:
    print(e)

data_population.head()



,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,POP,TOT,AGRWTH,A,1951,2.971132,NaN
1,AUS,POP,TOT,AGRWTH,A,1952,2.550554,NaN
2,AUS,POP,TOT,AGRWTH,A,1953,2.070283,NaN
3,AUS,POP,TOT,AGRWTH,A,1954,1.942078,NaN
4,AUS,POP,TOT,AGRWTH,A,1955,2.372448,NaN


In [37]:
for index, data in data_population.iterrows():
    print(f"""valeur de col n°1 : {data["LOCATION"]} """, f"""valeur de col n°2 : {data["Value"]}""", 
    f"""valeur de col n°3 : {data["TIME"]}""")

valeur de col n°1 : AUS  valeur de col n°2 : 2.97113233154413 valeur de col n°3 : 1951
valeur de col n°1 : AUS  valeur de col n°2 : 2.55055392616693 valeur de col n°3 : 1952
valeur de col n°1 : AUS  valeur de col n°2 : 2.07028310079315 valeur de col n°3 : 1953
valeur de col n°1 : AUS  valeur de col n°2 : 1.94207797806087 valeur de col n°3 : 1954
valeur de col n°1 : AUS  valeur de col n°2 : 2.37244756022923 valeur de col n°3 : 1955
valeur de col n°1 : AUS  valeur de col n°2 : 2.4544278617781 valeur de col n°3 : 1956
valeur de col n°1 : AUS  valeur de col n°2 : 2.27786324332927 valeur de col n°3 : 1957
valeur de col n°1 : AUS  valeur de col n°2 : 2.09746685753408 valeur de col n°3 : 1958
valeur de col n°1 : AUS  valeur de col n°2 : 2.1742664390799 valeur de col n°3 : 1959
valeur de col n°1 : AUS  valeur de col n°2 : 2.17374010580327 valeur de col n°3 : 1960
valeur de col n°1 : AUS  valeur de col n°2 : 2.26958637469586 valeur de col n°3 : 1961
valeur de col n°1 : AUS  valeur de col n°2 : 

### Insertion du fichier csv 'population' dans la table 'populations' de la BDD

In [38]:
for index, data in data_population.iterrows():
    datedf = datetime.datetime(int(data['TIME']),1,1).strftime("%Y%m%d")
    query = f"""INSERT INTO populations (id, location, time, value) """
    query += f"""VALUES ({index+1},"{data["LOCATION"]}", "{datedf}", "{data['Value']}") """
    query += f"""ON DUPLICATE KEY UPDATE id = {index+1} """
    cursor.execute(query)

bdd.commit()

<center><img src="capture_populations.png" width="500" height="500"></center>

### Mise en place du SELECT / JOIN permettant de remplacer le code du pays par son nom dans la table 'populations'

In [39]:
query1 = """SELECT populations.id, countries.name, populations.time, populations.value FROM populations INNER JOIN countries ON populations.location = countries.code"""
cursor.execute(query1)

result = cursor.fetchall()

### Affichage du dataframe correspondant à la table 'populations' avec le nom du pays à la place du code du pays

In [41]:
populations_final = result

df = pd.DataFrame.from_records(
    populations_final, columns=['id', 'name', 'time', 'value'])
print(df)

          id       name        time     value
0          1  Australia  1951-01-01  2.971130
1          2  Australia  1952-01-01  2.550550
2          3  Australia  1953-01-01  2.070280
3          4  Australia  1954-01-01  1.942080
4          5  Australia  1955-01-01  2.372450
...      ...        ...         ...       ...
12332  15628  Singapore  2016-01-01  0.790967
12333  15629  Singapore  2017-01-01  0.819538
12334  15630  Singapore  2018-01-01  0.718317
12335  15631  Singapore  2019-01-01  0.799292
12336  15632  Singapore  2020-01-01  0.447096

[12337 rows x 4 columns]
